# 📘 Sentiment Analysis mit RoBERTa und MongoDB

Dieses Notebook erlaubt es dir, Texte aus MongoDB zu laden, mit einem RoBERTa-Modell das Sentiment zu analysieren und die Ergebnisse abzuspeichern.

Am Ende kann das Ganze in ein Python Script überführt werden.

## 1. 🔧 Setup & Imports

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from pymongo import MongoClient

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.2.4-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-77.0.3-py3-none-any.whl.metadata (6.6 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 25.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 32.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 36.5 MB/s eta 0:00:00
Using cached numpy-2.2.4-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl (171 kB)
Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl (284 kB)
   ━━━━━━━━━━━━━━

/Users/yanickfischer/Documents/vsCode/MDM/project1/ukraineBias/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 🧱 Verbindung zur MongoDB

In [ ]:
# MongoDB URI hier einfügen
MONGO_URI = "mongodb+srv://<user>:<pass>@<cluster>.mongodb.net"
DB_NAME = "deine_datenbank"
COLLECTION_NAME = "deine_collection"

client = MongoClient(MONGO_URI)
collection = client[DB_NAME][COLLECTION_NAME]

## 3. 🤗 RoBERTa Modell laden

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
labels = ['negative', 'neutral', 'positive']

## 4. 🔍 Beispielhafte Sentiment-Inferenz

In [ ]:
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = torch.argmax(logits).item()
    return labels[predicted_class_id]

# Beispiel testen
get_sentiment("Ich liebe dieses Produkt!")

## 5. 🧪 Anwendung auf MongoDB-Daten

In [ ]:
for doc in collection.find().limit(5):
    text = doc.get("text", "")
    sentiment = get_sentiment(text)
    print(f"Text: {text}\n→ Sentiment: {sentiment}\n")

## 6. 💾 Sentiment in MongoDB speichern

In [ ]:
# Optional: Sentiment direkt in die Datenbank schreiben
for doc in collection.find():
    text = doc.get("text", "")
    sentiment = get_sentiment(text)
    collection.update_one({"_id": doc["_id"]}, {"$set": {"sentiment": sentiment}})